# Preprocess AIS Data Set 2 and derieve XY positionings for vessels in Hamburg harbour 

### Purpose: Convert all geopositions of 6 different AIS ship tracks to cartesian coordinates to make transient (i.e. non-stationary) CFD simulations possible in which all ships move within the computational domain.

### Challenge: The AIS tracks are partly longer than the CFD computational domain and each track enters or leaves the domain on different directions and times. Each vessel track characteristics needs to be considered. 

#### Requirements: <br>- build unified time frame (4 hours envisaged) and store all positions of each vessel in separete x,y positioning files (x & y separately). <br>- positions must be relative to each vessel's start coordinate

#### 1st data sighting HGE MO 13 Aug 2018
#### Productive version HGE THU 16 Aug 2018

no visualisation in this notebook

In [1]:
import os
import errno
import csv
import numpy as np
import math
import datetime
from mpl_toolkits.basemap import Basemap
#import folium

## 1) Set the order

### Aligns from the beginning of a vessel movement or 10:00:

ID 1
- 10:00:00 (- 11:08:30)

ID 2
- 10:00:00 (- 13:17:30)

ID 3
- 10:28:00 (- 11:13:30) 

ID 4
- 10:51:30 (- 13:25:30)

ID 5
- 12:26:30 (- 13:59:30)

ID 6
- 13:04:30 (- 13:59:30)

In [2]:
vessel_id = ['add_id','add_id','add_id',\
             'add_id','add_id','add_id']

inpath = 'add_path_to/csv/'

outpath = 'add_path_/'

### General defintions wrt mapping:

#### _Define map projection to establish conversion from cartesian XX,YY grid:_

In [3]:
m = Basemap(projection='tmerc',resolution='i',
            llcrnrlat=53., urcrnrlat=54.,
            llcrnrlon=8.5, urcrnrlon=10.5,
            lat_0=53.5,lon_0=10.) 

#### _set size of simulation domain (should be same size as region which is analysed):_

In [4]:
sim_domain_edge_length = 10000. # meter
sim_domain_half_width  = sim_domain_edge_length/2.

# Base coordinate of simulation domain (position of simulation domain centre point):
# this is the coord. from which in each direction (N,S,W,E) the domain half-width of 
# 5km begins. Means, entering or leaving the domain is calculated relative the the 
# vertical or horizonatalaxis that is intersecting this center point. In this case 
# about the middle of Blohm+Voss shipyard:
lon_sim_base = 9.961250
lat_sim_base = 53.539373

#### _Map the geographic base coordinate of simulation domain to global (projection specific) cartesian coordinates:_
This point is fixed and vessel independent.

In [5]:
xpt0_sim_base, ypt0_sim_base = m(lon_sim_base,lat_sim_base)
print('Cartesian base coordinate of simulation domain (9.961250/53.539373)  x/y:',\
      xpt0_sim_base, ypt0_sim_base)

Cartesian base coordinate of simulation domain (9.961250/53.539373)  x/y: 98133.64755970874 58975.38045677248


### Functions:

In [6]:
def vessel_read(filename):
    if not os.path.isfile(filename):
        raise FileNotFoundError(
            errno.ENOENT, os.strerror(errno.ENOENT), filename)
    else:
        print('read file :',filename)
        with open(filename) as csvfile:
            reader = csv.DictReader(csvfile)
            t_unix= [] # unix time stamp in ms
            x_geo = []
            y_geo = []
            for row in reader:
                t_unix.append(row['ts'])
                x_geo.append(row['x'])
                y_geo.append(row['y'])
        return  t_unix, x_geo, y_geo

In [7]:
# just for control time intervalls in incoming data - this adds no operational value
def inspect_time_intervals(t):
    # loop until n-1 because upwind differences applied:
    for idx,obj in enumerate(range(0,len(t)-1)):   
        dt = int(t[idx+1]) - int(t[idx])
        print('    ',idx,dt)

In [8]:
# converts date time string back into unix time since epoch in ms:
def time_string_to_unixtime(obj):
    dt_obj = datetime.datetime.strptime(obj,  '%Y-%m-%d %H:%M:%S') 
    millisec = dt_obj.timestamp() * 1000
    return int(millisec)


# converts datetime object (that is something different than above!!!) back 
# into unix time since epoch in ms:
def time_datetime_to_unixtime(obj):
    millisec = obj.timestamp() * 1000
    return int(millisec)

In [100]:
# not all direction filters implemented yet
# further modularisation possible (rel. hardwired so far)
# Potentially a python class is better suiting the purpose of a filter
# Note that each AIS track shows very individual characteristics - hence each filter 
# applies very different methodologies. 
# Only track 1 & 4 have enough similarities that they can be processed by the same method.

# idea: loop through time 
#       each dt has a unique x & y position
#       compare & calc. difference vessel pos to overall base-pos. (=centre) of simulation 
#       domain partly look 2min ahead or backward for modifying track positions



# ID 6  enters westward (coming from east), hence in negative x direction
#       northbound when entering domain (this is not necessarily the overall/average 
#       direction!)
#       Here:
#       - the initial vessel postion must NOT be reset its close enough (-4819m off 
#         sim-base)
#       - hence, not requiring recalculation of all dispalcements relative to reset 
#         initial position
#
def enter_from_east_northbound(t_unmod, x_pos_unmod, y_pos_unmod,\
                               dx_unmod, dy_unmod, x_ini, y_ini):    # ID 6 (green)
    
    print('    Track filtering enter_from_east_northbound')
        
    # loop through reversed X direction 
    
    x_mod_x_scan = x_pos_unmod.copy()   
    y_mod_y_scan = y_pos_unmod.copy()  
    
    dx_mod_x_scan = dx_unmod.copy()      
    dy_mod_y_scan = dy_unmod.copy()   
    
    
    # calc difference vessel start pos to overall base-pos. (=centre) of simulation domain:
    
    dx_vessel_start_to_sim_centre = xpt0_sim_base - x_ini
    
    dy_vessel_start_to_sim_centre = ypt0_sim_base - y_ini
    
     
    #!!!!!!!!!!!!! DEBUGGING !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    print('xpt0_sim_base: ',xpt0_sim_base,'  x_ini: ',x_ini)
    print('ypt0_sim_base: ',ypt0_sim_base,'  y_ini: ',y_ini)

    print('dx_vessel_start_to_sim_centre: ',dx_vessel_start_to_sim_centre,\
          '  T/F: ',abs(dx_vessel_start_to_sim_centre) > sim_domain_half_width)
    print('dy_vessel_start_to_sim_centre: ',dy_vessel_start_to_sim_centre,\
          '  T/F: ',abs(dy_vessel_start_to_sim_centre) > sim_domain_half_width)

    
    
    # is this position outside simulation domain?
    # if so, raise error because this shouldn't happen here (scan for leaving domain only)
    
    if abs(dx_vessel_start_to_sim_centre) > sim_domain_half_width:
        raise ValueError('Vessel initial X position outside simulation domain: {}'.format(dx_vessel_start_to_sim_centre))
    else:
        print('X direction distance of vessel initial position:',dx_vessel_start_to_sim_centre)
        
    if abs(dy_vessel_start_to_sim_centre) > sim_domain_half_width:
        # since in this case its only about 400m I assume its still within the 
        # computational domain which is slightly bigger (circular) than the simulation or
        # analysis domain and we do not raise the error here
        print('Vessel initial Y position outside simulation domain: ',\
              dy_vessel_start_to_sim_centre)
    else:
        print('Vessel initial Y position inside simulation domain:',\
              dy_vessel_start_to_sim_centre)

    
    
    #- X scan
    for idt, t in enumerate(t_unmod): 

        # for ID 6 that way around (positive distance from sim-base) 
        # which is getting smaller with time & vessel motion (at beginning of track)
        
        dx_distance_to_sim_base =  x_pos_unmod[idt] - xpt0_sim_base         
                 
            
    #- Y scan
    for idt, t in enumerate(t_unmod):
        
        dy_distance_to_sim_base =  y_pos_unmod[idt] - ypt0_sim_base
        
   
    # correct just the very last displacements (otherwise it jumps back to inital pos):
    
    dx_mod_x_scan[ len(dx_mod_x_scan)-1 ] = dx_mod_x_scan[ len(dx_mod_x_scan)-2 ]
    dy_mod_y_scan[ len(dy_mod_y_scan)-1 ] = dy_mod_y_scan[ len(dy_mod_y_scan)-2 ]
    
    return x_mod_x_scan, y_mod_y_scan, dx_mod_x_scan, dy_mod_y_scan




# ID 5: enters eastward (coming from west), hence in positve x direction 
#       here:
#       - the initial vessel postion must be set to a position near simulation domain
#       - requires recalculation of all dispalcements relative to reset initial position
#       - Y scan is necessary because original AIS vessel starting position's y > y_base+5km
#       - the time stamp of the new intial position is determined from the latest occurence
#         of entering the sim-domain (of the original track) in either X or Y direction
#       -> actually in this case (ID5) in Y it reaches sim-domain in time step 306 whereas 
#         in X it reaches sim-domain at time step 353 -> hence 353 determines start pos 
#         and displacements 
#
def enter_from_west_southbound(t_unmod, x_pos_unmod, y_pos_unmod,\
                                  dx_unmod, dy_unmod, x_ini, y_ini):  # ID 5  (light blue)
    
    print('    Track filtering enter_from_west_southbound')
        
    # loop in positive X direction 
    
    x_mod_x_scan = x_pos_unmod.copy()    
    y_mod_y_scan = y_pos_unmod.copy()   
    
    dx_mod_x_scan = dx_unmod.copy()     
    dy_mod_y_scan = dy_unmod.copy()     
    
    
    # calc difference vessel start pos to overall base-pos. (=centre) of simulation domain:
    
    dx_vessel_start_to_sim_centre = xpt0_sim_base - x_ini
    
    dy_vessel_start_to_sim_centre = ypt0_sim_base - y_ini
    
     
    print('xpt0_sim_base: ',xpt0_sim_base,'  x_ini: ',x_ini)
    print('ypt0_sim_base: ',ypt0_sim_base,'  y_ini: ',y_ini)

    print('dx_vessel_start_to_sim_centre: ',dx_vessel_start_to_sim_centre,\
          '  T/F: ',abs(dx_vessel_start_to_sim_centre) > sim_domain_half_width)
    print('dy_vessel_start_to_sim_centre: ',dy_vessel_start_to_sim_centre,\
          '  T/F: ',abs(dy_vessel_start_to_sim_centre) > sim_domain_half_width)

    
    # is this position outside simulation domain?
    # if so, raise error because this shouldn't happen here (we scan for leaving domain only)
    
    if abs(dx_vessel_start_to_sim_centre) > sim_domain_half_width:
        print('Vessel initial X position outside simulation domain: ',\
              dx_vessel_start_to_sim_centre)
    else:
        print('X direction distance of vessel initial position:',\
              dx_vessel_start_to_sim_centre)
        
    if abs(dy_vessel_start_to_sim_centre) > sim_domain_half_width:
        print('Vessel initial Y position outside simulation domain: ',\
              dy_vessel_start_to_sim_centre,\
              'SPECIAL FILTERING NECESSARY !!!!!!!!!')
    else:
        print('Vessel initial Y position inside simulation domain:',\
              dy_vessel_start_to_sim_centre)

    
    # scan for positions ahead:
    
    t_offset = 4 # for ID 2min are likely 4 timesteps
    
    
    #- X scan
    for idt, t in enumerate(t_unmod[:-t_offset]):               
        
        dt_4_pos_backward = t_unmod[idt + t_offset]
        
        
        
        # X modification case ID 5:
        # comes from smaller x values going to bigger x values in positive 
        # direction!
        
        # define vessel stopping criterium: if vessel reaches sim-domain edge, 
        # all values up to the end of overall time frame are replicated from 
        # 2 min backward from the edge position (and loop is interrupted):
        
        
        dx_distance_to_sim_base =  x_pos_unmod[idt] - xpt0_sim_base        
         
        
        if dx_distance_to_sim_base > (-sim_domain_half_width):     
            
            
            # should interrupt at idx 355 - save value of 4 steps backward 
            # as new intial position: (2nd criterium here is that last time 
            # step's value is not fullfilling the 1st criterium)
            
            if (x_pos_unmod[idt-1] - xpt0_sim_base) < (-sim_domain_half_width):
                
                xpt0_vessel_reset = x_mod_x_scan[idt - t_offset]   
                
            
            # modify range from 2min backward of this position up the 
            # beginning of list (!!!):
        
            x_mod_x_scan[:idt - t_offset]  = x_mod_x_scan[idt - t_offset]  
            dx_mod_x_scan[:idt - t_offset] = dx_mod_x_scan[idt - t_offset]
            
            
            # keep index with offset (!!) to compare with Y scan
            
            idt_x = idt - t_offset
           
             
            # to ensure that the position 4 time steps away is not too far away 
            # (sometimes spacing is not 30sec but up to 120sec or even more), 
            # calculate the time difference between the two points (in seconds):
            
            t_difference_this_scan = (t_unmod[idt - t_offset] - t ) /1000.
            
            print('   delta time is : ',t_difference_this_scan,' sec')
            
            if abs(t_difference_this_scan) > 120.:
                raise ValueError('Vessel stop time difference exceeds 120sec: {}'.format(t_difference_this_scan))
    
            break
        
    #- Y scan - very special treatment here because of coarse & irrregular AIS positioning 
    for idt, t in enumerate(t_unmod):
       
        # Y modification case ID 5:
        # comes from bigger y values going to smaller y values in negative direction!
        
        
        dy_distance_to_sim_base =  y_pos_unmod[idt] - ypt0_sim_base
        
    
        if dy_distance_to_sim_base < sim_domain_half_width: 
            
            if idt < idt_x:
                print('ID 5 Y scan: found ini pos at {} earlier than X: {} '.format(idt,idt_x))
                print('-> Hence X index determines new intial position!! Interupt Y scan.')
                print('y_mod_y_scan[idt]   : ',y_mod_y_scan[idt])
                print('y_mod_y_scan[idt_x] : ',y_mod_y_scan[idt_x])
                            
                ypt0_vessel_reset = y_mod_y_scan[idt_x]       
            
                y_mod_y_scan[:idt_x]  = y_mod_y_scan[idt_x]   
                dy_mod_y_scan[:idt_x] = dy_mod_y_scan[idt_x] 
                
            else:
                raise ValueError('ID 5: Error resetting start pos in Y direction.')

            
            print('Carefully initialise plume/engine as f(t) here (in simulation)!')
            
            break
    
    
    print('Resetted initial X position {} at time step {}'.format(xpt0_vessel_reset,idt_x))
    print('Resetted initial Y position {} at time step {}'.format(ypt0_vessel_reset,idt_x))
    
    #- Recalculation of all displacements relative to reset initial position:
    
    dx_reset = [0.]  
    dy_reset = [0.]

    for idx, obj in enumerate(t_unmod):
        if idx >0:
        
            dxi = x_mod_x_scan[idx] - xpt0_vessel_reset
            dyi = y_mod_y_scan[idx] - ypt0_vessel_reset
                
            dx_reset.append(dxi)
            dy_reset.append(dyi)
            
            
     #- round values and format to ensure 4 decimals are returned
    
    dx_reset_rnd = ['%.4f' % np.around(val,4) for val in dx_reset]
    dy_reset_rnd = ['%.4f' % np.around(val,4) for val in dy_reset] 
   
    
   
    return x_mod_x_scan, y_mod_y_scan, dx_reset_rnd, dy_reset_rnd




# ID 3: moves out eastward ("right side"), hence in positve x direction
def leave_on_east_side_southbound(t_unmod, x_pos_unmod, y_pos_unmod,\
                                  dx_unmod, dy_unmod, x_ini, y_ini):  # ID 3 (purple)
    
    print('    Track filtering leave_on_east_side_southbound')
        
    # loop in positive X direction 
    
    x_mod_x_scan = x_pos_unmod.copy()   
    y_mod_y_scan = y_pos_unmod.copy()  
    
    dx_mod_x_scan = dx_unmod.copy()    
    dy_mod_y_scan = dy_unmod.copy()     
    
    
    # calc difference vessel start pos to overall base-pos. (=centre) of simulation domain:
   
    dx_vessel_start_to_sim_centre = x_ini - xpt0_sim_base   
    
    dy_vessel_start_to_sim_centre = y_ini - ypt0_sim_base
    
     
    print('xpt0_sim_base: ',xpt0_sim_base,'  x_ini: ',x_ini)
    print('ypt0_sim_base: ',ypt0_sim_base,'  y_ini: ',y_ini)

    print('dx_vessel_start_to_sim_centre: ',dx_vessel_start_to_sim_centre,\
          '  T/F: ',abs(dx_vessel_start_to_sim_centre) > sim_domain_half_width)
    print('dy_vessel_start_to_sim_centre: ',dy_vessel_start_to_sim_centre,\
          '  T/F: ',abs(dy_vessel_start_to_sim_centre) > sim_domain_half_width)
    
    
    if abs(dx_vessel_start_to_sim_centre) > sim_domain_half_width:
        raise ValueError('Vessel initial X position outside simulation domain: {}'.format(dx_vessel_start_to_sim_centre))
    else:
        print('X direction distance of vessel initial position:',dx_vessel_start_to_sim_centre)
        
    if abs(dy_vessel_start_to_sim_centre) > sim_domain_half_width:
        print('Vessel initial Y position outside simulation domain: ',dy_vessel_start_to_sim_centre)
    else:
        print('Vessel initial Y position inside simulation domain:',dy_vessel_start_to_sim_centre)
    
    
    
    # scan for positions ahead:
    
    t_offset = 4 # 2min are likely 4 timesteps a 30sec
    
    
    # loop until 4th last time step
    for idt, t in enumerate(t_unmod[:-t_offset]):                      
        
        dt_4_pos_ahead = t_unmod[idt + t_offset]
        
        
        # in this case ID 3:
        # if vessel moves out of domain it comes from smaller x values going to bigger x 
        # values (in positive direction!)
        
        # define vessel stopping criterium: if vessel reaches sim-domain edge, all values up to 
        # the end of overall time frame are replicated from 2 min away from the edge position 
        # (and loop is interrupted):
        
        
        dx_distance_to_sim_base =  x_pos_unmod[idt] - xpt0_sim_base 
        
      
        if dx_distance_to_sim_base > sim_domain_half_width:
        # should interrupt at idx 143:
            
            # modify range from 2min away of this position until end of list:
            
            x_mod_x_scan[idt + t_offset:]  = x_mod_x_scan[idt + t_offset]
            dx_mod_x_scan[idt + t_offset:] = dx_mod_x_scan[idt + t_offset]
            
            # also set Y displacement (otherwise vessel would "jump" back in Y during simulation):
            
            y_mod_y_scan[idt + t_offset:]  = y_mod_y_scan[idt + t_offset]
            dy_mod_y_scan[idt + t_offset:] = dy_mod_y_scan[idt + t_offset]
             
            # to ensure that the position 4 time steps away is not too far away (sometimes 
            # spacing is not 30sec but up to 120sec or even more), calculate the time 
            # difference between the two points (in seconds):
            
            t_difference_this_scan = (t_unmod[idt + t_offset] - t ) /1000.
            
            print('   delta time is : ',t_difference_this_scan,' sec')
            
            if t_difference_this_scan > 120.:
                raise ValueError('Vessel stop time difference exceeds 120sec: {}'.format(t_difference_this_scan))
            
            break
    
    return x_mod_x_scan,y_mod_y_scan, dx_mod_x_scan, dy_mod_y_scan



# ID 1 & 4:  moves out westward ("left side"), in negative x direction 
# (and positive in Y)
def leave_on_west_side_northbound(t_unmod, x_pos_unmod, y_pos_unmod,\
                                  dx_unmod, dy_unmod, x_ini, y_ini):  # ID 1 & 4 (red)
    
    print('    Track filtering leave_on_west_side_northbound')
    
    # loop through reversed X direction 
    
    x_mod_x_scan = x_pos_unmod.copy()   
    y_mod_y_scan = y_pos_unmod.copy()    
    
    dx_mod_x_scan = dx_unmod.copy()     
    dy_mod_y_scan = dy_unmod.copy()      
    
    
    # calc difference vessel start pos to overall base-pos. (=centre) of simulation domain:
    
    dx_vessel_start_to_sim_centre = xpt0_sim_base - x_ini
    
    dy_vessel_start_to_sim_centre = ypt0_sim_base - y_ini
    
    
    # is this position outside simulation domain?
    # if so, raise error because this shouldn't happen here (we scan for leaving domain only)
    
    if abs(dx_vessel_start_to_sim_centre) > sim_domain_half_width:
        raise ValueError('Vessel initial X position outside simulation domain: {}'.format(dx_vessel_start_to_sim_centre))
    else:
        print('X direction distance of vessel initial position:',dx_vessel_start_to_sim_centre)
        
    if abs(dy_vessel_start_to_sim_centre) > sim_domain_half_width:
        print('Vessel initial Y position outside simulation domain: ',dy_vessel_start_to_sim_centre)
    else:
        print('Vessel initial Y position inside simulation domain:',dy_vessel_start_to_sim_centre)

    
    
    # scan for positions ahead:
    
    t_offset = 4 # 2min are likely 4 timesteps a 30sec
    
    
    
    for idt, t in enumerate(t_unmod[:-t_offset]):                   
        
        dt_4_pos_ahead = t_unmod[idt + t_offset]
        
              
        
        dx_distance_to_sim_base =  xpt0_sim_base - x_pos_unmod[idt]
        
       # print(idt, t,' X check:',xpt0_sim_base,x_pos_unmod[idt],\
       #       dx_unmod[idt],dx_distance_to_sim_base,\
       #       '  Y check:',dy_unmod[idt]
       #      )
        
        
        if dx_distance_to_sim_base > sim_domain_half_width: 
        # should interrupt at idx 75 for ID 1:
            
            # modify range from 2min away of this position until end of list:
            
            x_mod_x_scan[idt + t_offset:]  = x_mod_x_scan[idt + t_offset]
            dx_mod_x_scan[idt + t_offset:] = dx_mod_x_scan[idt + t_offset]
            
            # also set Y displacement (otherwise vessel would "jump" back in Y during 
            # simulation):
            
            y_mod_y_scan[idt + t_offset:]  = y_mod_y_scan[idt + t_offset]
            dy_mod_y_scan[idt + t_offset:] = dy_mod_y_scan[idt + t_offset]
          
            t_difference_this_scan = (t_unmod[idt + t_offset] - t ) /1000.
                        
            if t_difference_this_scan > 120.:
                raise ValueError('Vessel stop time difference exceeds 120sec: {}'.format(t_difference_this_scan))
            
            break
    
    return x_mod_x_scan,y_mod_y_scan, dx_mod_x_scan, dy_mod_y_scan


    
# ID 2:  not entering or leaving sim-domain - but stop positions need to be corrected:
def set_last_pos_dxdy(t_unmod, x_pos_unmod, y_pos_unmod,\
                      dx_unmod, dy_unmod, x_ini, y_ini):        # ID 2
    print('    Setting stop positions.')
    
    
    dx_mod_x_scan = dx_unmod.copy()     
    dy_mod_y_scan = dy_unmod.copy()      
    
   
    
    # scan for detecting stop position:
    for idt, t in enumerate(t_unmod):  #[:-1]
        
        # stop pos is characterised as no deviation from mean from to stop to end of time series!
        # should detect idt 132
        
        avg_x_eots = np.mean(x_pos_unmod[idt:])
        avg_y_eots = np.mean(y_pos_unmod[idt:])
                
        
        # compare truncated values to avoid numerical precision problems:

        if '%.6f' % x_pos_unmod[idt] == '%.6f' % avg_x_eots and \
           '%.6f' % y_pos_unmod[idt] == '%.6f' % avg_y_eots:  
       
            print('ID 2: stop position detected at :',\
                  idt,t_unmod[idt],x_pos_unmod[idt],y_pos_unmod[idt])
            
            dx_mod_x_scan[idt:] = dx_mod_x_scan[idt] 
            dy_mod_y_scan[idt:] = dy_mod_y_scan[idt]
            
            break
        else:   
            if idt == len(t_unmod)-1:
                raise ValueError('ID 2: No vessel final stop position detected')

    
    return x_pos_unmod, y_pos_unmod, dx_mod_x_scan, dy_mod_y_scan

In [101]:
def write_files(filelist, varlist):

    for item,obj in enumerate(filelist):
        with open(outpath + obj,'w') as f:
            writer = csv.writer(f)
            for val in varlist[item]:
                writer.writerow(val)

### General definitions wrt time:

In [102]:
# start 5 min to 10:00 in order to establish 1st vessels exhaust plume

t_initial = datetime.datetime(2018, 6, 28, 9, 55)
print ('Initial time :',t_initial.isoformat(' '),'   unix time stamp:',\
       time_datetime_to_unixtime(t_initial))


# define fixed stop time for vessel motion:

t_stop_overall = datetime.datetime(2018, 6, 28, 14, 00)
print ('Overall stop time: ',t_stop_overall,type(t_stop_overall),|
       t_stop_overall.strftime('%Y-%m-%d %H:%M:%S'))
print ('                                      unix time stamp:',|
       t_stop_overall.timestamp() * 1000)


# define time window for searching vessel motion adjustment in order 
# not allowing a vessel to leave the computational domain no more than 
# two minutes away (maybe 2min is too much)
track_correction_time_window = 2.*60.*1000.  # milliseconds

Initial time : 2018-06-28 09:55:00    unix time stamp: 1530172500000
Overall stop time:  2018-06-28 14:00:00 <class 'datetime.datetime'> 2018-06-28 14:00:00
                                      unix time stamp: 1530187200000.0


### Process all files and tracks:

In [103]:
running_order = 1          #... human readable

for id in vessel_id:
    
    ## 1) read each AIS file
    ##    Code change (compared to prepare_xy_positionings_displacements_03.ipynb): renamed 
    ##    x_geo/y_geo into lon_ais, lat_ais (in def above old names still used)
    
    t_unix, lon_ais, lat_ais = vessel_read(inpath+'vesselmove_'+ id +'_dt20180629_v0.csv')  
    
    
    ## 2) set base coords and time:  (little code change to simplify handling)

    lon_ais_0 = lon_ais[0]
    lat_ais_0 = lat_ais[0]
    base_time = t_unix[0]

    print(id,':   ',t_unix[0],lon_ais[0],lat_ais[0])
    
    
    
    ## 3) Convert time to store in seconds since begin (list comprehension):
    ##    minor code change: datetime.datetime now
    
    t_hr=[datetime.datetime.fromtimestamp(int(t)/1000).strftime('%Y-%m-%d %H:%M:%S') for t in t_unix]
        

    ## 4) check if there are time intervalls != 30sec (important for temporal 'mapping'):
    #inspect_time_intervals(t_unix)
    
    
    
    ## 5) temporal mapping of track's time frame into the overall time frame of simulation:
    ##    this is not a true mapping - here just the period/timeseries up to t_initial is added 
    ##    at the beginning, prior to starting time of vessel and, respectively, and at the end 
    ##    we extend the time to t_stop_overall
    
    #   5.1) calc difference between start time of vessel and initial time (in ms):

    t_delta_begin_ms = int(base_time) - time_datetime_to_unixtime(t_initial)
        
    print('Vessel ID: ',id,':   time to add prior to track: ',t_delta_begin_ms,\
          'ms (',t_delta_begin_ms/1000/60,' min)')

    
    #   5.2) calc difference between last time stamp in data set and overall stop time (in ms):
    

    ## NOT USED LATER (direct defintiuon of time array below)
    t_delta_subsequently_ms = time_datetime_to_unixtime(t_stop_overall) - int(t_unix[len(t_unix)-1])
    
    
    print('Vessel ID: ',id,':   time to add subsequently to track: ',t_delta_subsequently_ms,\
          'ms (',t_delta_subsequently_ms/1000/60,' min)')

    
    #   5.3) set time frames to be added in 30sec intervalls:
        
    time_frame_prior_unixtime = np.arange(
                                        time_datetime_to_unixtime(t_initial), 
                                        time_datetime_to_unixtime(t_initial) + t_delta_begin_ms, 
                                        30000)
    
    print('Initial time                 (t_initial)  :',time_datetime_to_unixtime(t_initial))
    print('Vessels start time           (base_time)  :',base_time)

       
    # here one need to add 30 sec offset to last time step in data file AND 30sec to end of array 
    time_frame_subsequent_unixtime = np.arange(
                                        int(t_unix[len(t_unix)-1]) + 30000,        # ms !!!
                                        int(t_stop_overall.timestamp() * 1000 + 30000), 
                                        30000)
    
    print('Vessels last time steps time (t_unix last):',int(t_unix[len(t_unix)-1]))
    print('Overall stop time            (t_initial)  :',t_stop_overall.timestamp() * 1000)

   
    
    ## 6) mapping of each vessel track's initial position: geo-coords --> global cartesian 
    #    (better: projection region cartesian)
    ##   Note that lon,lat can be scalars, lists or numpy arrays
    
    xpt0_vessel,ypt0_vessel = m(lon_ais_0,lat_ais_0)
    
       
    
    ## 7) convert all positions to global cartesian coords (cartesian mapping of all 
    #   track positions):
    ##    for better debuggin no list comprehension here
    ##    (code changed using lon_ais, lat_ais)
    
    x_glob_cartesian = []
    y_glob_cartesian = []

    
    print('* * * Cartesian mapping of all track positions:')
    
    for idx, obj in enumerate(lon_ais):

        xpt,ypt = m(lon_ais[idx],lat_ais[idx])
    
        x_glob_cartesian.append(xpt)
        y_glob_cartesian.append(ypt)
    
   
    # store last position to fill up overall time frame should the vesser stop before 
    # (which is the case for all vessels)
    # Note: this is a value corresponding to last entry of vessel track file!
    
    xpt_last_vessel = x_glob_cartesian[len(x_glob_cartesian)-1]
    ypt_last_vessel = y_glob_cartesian[len(y_glob_cartesian)-1]
    
    print('Global cartesian coords:')
    print('  Cartesian base coordinate of simulation domain (9.961250/53.539373)  x/y:',\
          xpt0_sim_base, ypt0_sim_base)  
    print('  1st pos of vessel X: ',xpt0_vessel,'  last pos X: ',xpt_last_vessel)
    print('  1st pos of vessel Y: ',ypt0_vessel,'  last pos Y: ',ypt_last_vessel)

    
    
    ## 8) calculate change from each position to REFERENCE position as delta_x, delta_y:
    ##    note 1: reference is vessels initial position! 
    ##    note 2: resulting vectors do not have entries for start position! --> first position's 
    #             displacement is zero!!! this nereds to be taken into account when later fill
    #             up the positions for the whole time frame!

    dx = []
    dy = []

    distances = [] # from origin !!!

    for idx, obj in enumerate(lon_ais):
        if idx >0:
        
            dxi = x_glob_cartesian[idx] - xpt0_vessel 
            dyi = y_glob_cartesian[idx] - ypt0_vessel 
                
            dx.append(dxi)
            dy.append(dyi)
        
            # distance according to pythagoras:
            distances.append(math.sqrt(dxi**2 + dyi**2))
        
            #print(idx,dxi,'  ',dyi,'  ',math.sqrt(dxi**2 + dyi**2))   
    
    
    ## 9) calc. max deviations from origin to later diefine size of simulation domain:
    ##    Note: when a ship is moving too far, it will be stopped when it leaves a core 
    ##          simulation domain - that is the same domain that is later used in 
    #           postprocessing
    ##
    ##    These deviations are stored in separete file
    ##
    
    min_x_displacement_total = min(dx)
    max_x_displacement_total = max(dx)
    min_y_displacement_total = min(dy)
    max_y_displacement_total = max(dy)
    
    print('  min/max deviations:  in X:',min(dx),max(dx),'  in Y:',min(dy),max(dy))
    
    
    
    ## 10) round numbers
    
    #dx_rnd = [round(val,4) for val in dx]
    #dy_rnd = [round(val,4) for val in dy]
     
    #- round values and format to ensure 4 decimals are returned
    
    dx_rnd = ['%.4f' % np.around(val,4) for val in dx]
    dy_rnd = ['%.4f' % np.around(val,4) for val in dy] 
    
    
    
    ## 11) concatenate prior, real, subsequent time frames:
    
    debug_this = False
    
    if debug_this:
        print('Times to be merged: \n',
            time_frame_prior_unixtime[len(time_frame_prior_unixtime)-1],'\n',
            t_unix[0],'\n',
            int(t_unix[len(t_unix)-1]),'\n',
            time_frame_subsequent_unixtime[0])
    
        print('Times to be merged: \n',
            type(time_frame_prior_unixtime[len(time_frame_prior_unixtime)-1]),'\n',
            type(int(t_unix[0])),'\n',
            type(  np.int64(int(t_unix[len(t_unix)-1]))  ),'\n',
            type(time_frame_subsequent_unixtime[0]))
   
        print('Times to be merged: \n',
            type(time_frame_prior_unixtime),'\n',
            type(t_unix),'\n',
            type(np.int64(t_unix)),'\n',       # should be converted to numpy int array
            type(time_frame_subsequent_unixtime))
    
    
    
    #   11.1) convert the native python list into ndarray containing np.int64
    #        (not converting vice versa because this ways is more convenient)
    
    tmp = [ np.int64(item)  for item in t_unix]
    
    time_frame_vesselmotion_unixtime = np.array(tmp)
    
    
    #   11.2) concatenate:
    
    t_track_merged = np.concatenate([time_frame_prior_unixtime, \
                                     time_frame_vesselmotion_unixtime, \
                                     time_frame_subsequent_unixtime])
  
    
    
    
        
    ## 12) position the vessel in frames prior and subsequently to given track:
    
    #  12.1) positions in cartesian coords - needed to later correct for simulation  
    #        if vessel goes to far or enters the simualtion domain: 
    
    #  12.1.1) set starting point-filled arrays in length of the two artifical 
    #          time frames, apply to both directions    
    
    x_pos_prior      = np.empty(len(time_frame_prior_unixtime))
    x_pos_prior.fill(xpt0_vessel)

    y_pos_prior      = np.empty(len(time_frame_prior_unixtime))
    y_pos_prior.fill(ypt0_vessel)
    
    x_pos_subsequent = np.empty(len(time_frame_subsequent_unixtime))
    x_pos_subsequent.fill(xpt_last_vessel)
    
    y_pos_subsequent = np.empty(len(time_frame_subsequent_unixtime))
    y_pos_subsequent.fill(ypt_last_vessel)
    

   

    #  12.1.2) concatenate
    
    x_pos_merged = np.concatenate([ x_pos_prior, x_glob_cartesian , x_pos_subsequent ])
    y_pos_merged = np.concatenate([ y_pos_prior, y_glob_cartesian , y_pos_subsequent ])   

    
    
    #  12.2) displacements dx & dy: 
    
    #  12.2.1) set 0-filled arrays in length of the two artifical time frames 
    #         (0== no displacement), apply to both directions
    
    xy_pos_prior      = np.zeros(len(time_frame_prior_unixtime))
    xy_pos_subsequent = np.zeros(len(time_frame_subsequent_unixtime))
    
   
    #  12.2.2) conversion list to ndarray
    tmpx = [ '%.4f' % np.around(np.float64(item),4)  for item in dx_rnd]
    tmpy = [ '%.4f' % np.around(np.float64(item),4)  for item in dy_rnd]
   
    dx_rnd_np = np.array(tmpx)
    dy_rnd_np = np.array(tmpy)
    
    
    # debugging:
    
    debug_this = False
    
    if debug_this:
        print('global cartesian X:')
        for idx, obj in enumerate(x_glob_cartesian[:-1]):
            if idx > 0:
                print('idx/x_glob_cartesian/dx : ',idx,obj,dx_rnd_np[idx])
        print('- - - - - -')

    
    
    #  12.2.3) concatenate
    #          note, the 0 displacement for vessels starting position (note that they are calculated 
    #          only for positions > intial!)
    #                                           
    dx_merged = np.concatenate([ xy_pos_prior, np.zeros(1), dx_rnd_np , xy_pos_subsequent ])
    dy_merged = np.concatenate([ xy_pos_prior, np.zeros(1), dy_rnd_np , xy_pos_subsequent ])
    

    
    ## 13) check if corrections on the vessel tracks are necessary 
    ##     e.g. stop the vessel when moving to far away or shift forward the position when entering the domain

    
    # track filter assignments (ID 2 not filtered): 
    filter_assignment = { 1 : leave_on_west_side_northbound,
                          4 : leave_on_west_side_northbound,
                          2 : set_last_pos_dxdy,
                          3 : leave_on_east_side_southbound,
                          5 : enter_from_west_southbound,
                          6 : enter_from_east_northbound,
                        }
    
    print('\n * * * Running Order: ',running_order)
    
    # pythonic case switch:
    x_pos_filtered, y_pos_filtered, dx_filtered , dy_filtered = filter_assignment[running_order](t_track_merged,\
                                                                   x_pos_merged, y_pos_merged,\
                                                                   dx_merged,    dy_merged,\
                                                                   xpt0_vessel,  ypt0_vessel)
    
    
    

    
    ## 14) zip relative time in sec and each modified list
    ##     this is done for both types of output: 
    ##     - the unmodified tracks & times
    ##     - and the simulation domain adapted tracks & times
    
    
    #  14.1) tracks:
    
    # unmodified:  x_pos_merged   & y_pos_merged
    # modified:    x_pos_filtered & y_pos_filtered
    
    t_x_pos_unmod , t_y_pos_unmod = [] , []
    t_x_pos_modif , t_y_pos_modif = [] , []

   
    # unmodified:
    for val in zip( t_track_merged ,x_pos_merged ):    # X
       #print(val)
        t_x_pos_unmod.append(val)
    
    for val in zip( t_track_merged ,y_pos_merged):     # Y
        t_y_pos_unmod.append(val)
    
    # modified:
    for val in zip( t_track_merged ,x_pos_filtered ):  # X
        #print(val)
        t_x_pos_modif.append(val)
    
    for val in zip( t_track_merged ,y_pos_filtered):   # Y
        t_y_pos_modif.append(val)
    
    
    
    #  14.2) displacements:
    
    # unmodified:  dx_merged   & dy_merged
    # modified:    dx_filtered & dy_filtered
    
    t_dx_unmod , t_dy_unmod = [] , []
    t_dx_modif , t_dy_modif = [] , []
    
    
    # unmodified:
    for val in zip( t_track_merged ,dx_merged ):    # X
        #print(val)
        t_dx_unmod.append(val)
    
    for val in zip( t_track_merged ,dy_merged):     # Y
        t_dy_unmod.append(val)
    
    # modified:
    for val in zip( t_track_merged ,dx_filtered ):  # X
        #print(val)
        t_dx_modif.append(val)
    
    for val in zip( t_track_merged ,dy_filtered):   # Y
        t_dy_modif.append(val)
    

    
    ## 15) write files
    
    fID = str(running_order)
    
    #  15.1) tracks:
    
    s1x = 't_x_pos_unmod_ID'
    s1y = 't_y_pos_unmod_ID'
    s2x = 't_x_pos_modif_ID'
    s2y = 't_y_pos_modif_ID'
   
    filelist = [ s1x+fID+'.dat' , s1y+fID+'.dat' , s2x+fID+'.dat' , s2y+fID+'.dat' ]
    
    varlist  = [ t_x_pos_unmod , t_y_pos_unmod , t_x_pos_modif , t_y_pos_modif ]
    
    write_files(filelist, varlist)

    
    #  15.2) displacements:
    
    s1x = 't_dx_unmod_ID'
    s1y = 't_dy_unmod_ID'
    s2x = 't_dx_modif_ID'
    s2y = 't_dy_modif_ID'
        
   
    filelist = [ s1x+fID+'.dat' , s1y+fID+'.dat' , s2x+fID+'.dat' , s2y+fID+'.dat' ]
    
    varlist  = [ t_dx_unmod , t_dy_unmod , t_dx_modif , t_dy_modif ]
    
    write_files(filelist, varlist)


    running_order +=1
    

read file : /Users/rene/Documents/Environmod_iMac/Projekte2/Bullemer_Harbour_Project/Data_Akio/Set2/Python/csv/vesselmove_id211443180_dt20180629_v0.csv
id211443180 :    1530172800000 9.955818 53.490818
Vessel ID:  id211443180 :   time to add prior to track:  300000 ms ( 5.0  min)
Vessel ID:  id211443180 :   time to add subsequently to track:  10290000 ms ( 171.5  min)
Initial time                 (t_initial)  : 1530172500000
Vessels start time           (base_time)  : 1530172800000
...
Vessels last time steps time (t_unix last): 1530176910000
Overall stop time            (t_initial)  : 1530187200000.0
* * * Cartesian mapping of all track positions:
Global cartesian coords:
  Cartesian base coordinate of simulation domain (9.961250/53.539373)  x/y: 98133.64755970874 58975.38045677248
  1st pos of vessel X:  97770.16883059918   last pos X:  82806.93333683792
  1st pos of vessel Y:  53571.616008122386   last pos Y:  61824.08800349565
  min/max deviations:  in X: -14963.235493761269 -40.97